In [ ]:
import os
import sys
import numpy as np
import k3d
import numpy as np
import cv2
from skimage.filters import gaussian
from scipy.interpolate import splev, splprep

In [ ]:
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/brainsharer/django_api')
sys.path.append(PATH)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "brainsharer.settings")
import django
django.setup()

In [ ]:
from neuroglancer.models import AnnotationSession
from brain.models import ScanRun, Section
from neuroglancer.annotation_session_manager import create_polygons

In [ ]:
def create_volume(polygons, width, height, z_length):
    color = 1
    volume = np.zeros((height, width, z_length), dtype=np.float64)
    print(f'volume shape: {volume.shape}')

    for z, points in polygons.items():
        #points = interpolate2d(points, 100)
        points = np.array(points)  - origin[:2]
        points = (points).astype(np.int32)
        volume_slice = np.zeros((height, width), dtype=np.uint8)
        volume_slice = cv2.polylines(volume_slice, [points], isClosed=True, color=color, thickness=1)
        volume_slice = cv2.fillPoly(volume_slice, pts = [points], color = color)
        volume[..., z] += volume_slice
    volume = np.swapaxes(volume, 0, 1)
    #volume = cv2.GaussianBlur(volume, (3,3), 1)
    #volume = gaussian(volume, 1, truncate=2)
    return volume.astype(np.uint8)

def create_volumeXXX(polygons, origin, section_size):
    volume = []
    color = 1
    for _, points in polygons.items():
        points = interpolate2d(points, len(points) * 2)
        # subtract origin so the array starts drawing in the upper top left
        points = np.array(points) - origin[:2]
        points = (points).astype(np.int32)
        volume_slice = np.zeros(section_size, dtype=np.uint8)
        volume_slice = cv2.polylines(volume_slice, [points], isClosed=True, color=color, thickness=1)
        volume_slice = cv2.fillPoly(volume_slice, pts=[points], color=color)
        volume.append(volume_slice)
    volume = np.array(volume).astype(np.float32)
    volume = np.swapaxes(volume,0,2)
    volume = gaussian(volume, 1, truncate=2)
    return volume

def get_origin_and_section_size(structure_contours):
    """Gets the origin and section size
    Set the pad to make sure we get all the volume
    """
    section_mins = []
    section_maxs = []
    for _, contour_points in structure_contours.items():
        contour_points = np.array(contour_points)
        section_mins.append(np.min(contour_points, axis=0))
        section_maxs.append(np.max(contour_points, axis=0))
    min_z = min([int(i) for i in structure_contours.keys()])
    min_x, min_y = np.min(section_mins, axis=0)
    max_x, max_y = np.max(section_maxs, axis=0)

    xspan = max_x - min_x
    yspan = max_y - min_y
    origin = np.array([min_x, min_y, min_z])
    # flipped yspan and xspan 19 Oct 2023
    section_size = np.array([yspan, xspan]).astype(int)
    return origin, section_size


def interpolate2d(points, new_len):
    """Interpolates a list of tuples to the specified length. The points param
    must be a list of tuples in 2d
    
    :param points: list of floats
    :param new_len: integer you want to interpolate to. This will be the new length of the array
    There can't be any consecutive identical points or an error will be thrown
    unique_rows = np.unique(original_array, axis=0)
    """

    pu = np.array(points, dtype=np.float64)
    indexes = np.unique(pu, axis=0, return_index=True)[1]
    points = np.array([points[index] for index in sorted(indexes)])

    tck, u = splprep(points.T, u=None, s=3, per=1)
    u_new = np.linspace(u.min(), u.max(), new_len)
    x_array, y_array = splev(u_new, tck, der=0)
    arr_2d = np.concatenate([x_array[:, None], y_array[:, None]], axis=1)
    return arr_2d


In [ ]:
# Full data structures look much better.
session_id = 3193
annotationSession = AnnotationSession.objects.get(pk=session_id)
scan_run = ScanRun.objects.get(prep=annotationSession.animal)

downsample_factor = 64
width = int(scan_run.width) // downsample_factor
height = int(scan_run.height) // downsample_factor
files = os.listdir(f"/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{annotationSession.animal}/preps/C1/thumbnail")
z_length = len(files)

polygons = create_polygons(annotationSession.annotation, scan_run.resolution, scan_run.zresolution, downsample_factor)
origin, section_size = get_origin_and_section_size(polygons)
print(f'origin={origin} section size={section_size}')
print(width, height, z_length)


In [ ]:
#volume = create_volume(polygons, origin, section_size)
volume = create_volume(polygons, width, height, z_length)
print(f'volume shape={volume.shape} dtype={volume.dtype}')

In [ ]:
plot = k3d.plot(background_color=255, grid_visible=False, lighting=0)
plot += k3d.volume(
    volume=volume.astype(np.float32),
    alpha_coef=1.0
)
plot.display()